In [1]:
# Libraries
import numpy as np
import pandas as pd
import math

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

In [2]:
# Running data - current day and next day
dat = pd.read_excel('running_data.xlsx')

In [3]:
dat.shape

(1972, 5)

In [4]:
dat.head()

,date,run_dist,notes,run_dist_1,notes_1
171,2016-10-10,10.0,"AM: Warmup (2 miles), 2 sets of (80, 50, 50), ...",10.0,"AM: Warmup (1.5 miles), Lift, Cooldown (2 mile..."
225,2016-11-21,10.0,"AM: Warmup (2 miles), 8x150 (Extensive Tempo),...",8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile..."
226,2016-11-22,8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile...",11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (..."
227,2016-11-23,11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (...",8.0,8 mile run. Not able to do squats or RDLs.
228,2016-11-24,8.0,8 mile run. Not able to do squats or RDLs.,10.5,"AM: Warmup (2 miles), 5x800 with 1:45 rest (Go..."


### Test on small dataset

In [36]:
#tr_small = pd.read_excel('rd_small_train.xlsx')
#t_small = pd.read_excel('rd_small_test.xlsx')
#v_small = pd.read_excel('rd_small_val.xlsx')

In [37]:
dat_small = pd.read_excel('rd_small_full.xlsx')

In [38]:
#frames = [tr_small, t_small, v_small]
#dat_small = pd.concat(frames)

In [39]:
#dat_small = dat_small.reset_index()
print dat.shape
dat_small.head()

(1972, 5)


,date,run_dist,notes,is_workout,difficulty,run_dist_1,notes_1
0,2016-12-29,8.00,"AM: 5 miles; PM: Warmup, lift, cooldown",0,0,9.0,"AM: Warmup (2 miles), 3x1000 with 3:30 recover..."
1,2015-07-26,10.00,Felt a little sore at the beginning of the run...,0,3,6.5,"6 mile run at around 6:40 pace, 6x100 strides."
2,2015-07-28,7.50,AM: 3 mile run at around 6:40 pace; PM: Warm-u...,1,2,5.0,5 mile run at around 6:30 pace.
3,2015-08-19,6.00,"6 mile run at 6:40 pace, 4x100 strides",0,3,7.0,"7 mile run at 6:45 pace, 4x100 strides"
4,2015-10-18,10.25,"10 mile run in 61:00 at Minuteman, 4x100 strid...",0,1,8.0,"AM: 3 miles; PM: Warmup (1.5 miles), 7x120 at ..."


In [40]:
dat_small.iloc[0]

date                                        2016-12-29 00:00:00
run_dist                                                      8
notes                   AM: 5 miles; PM: Warmup, lift, cooldown
is_workout                                                    0
difficulty                                                    0
run_dist_1                                                    9
notes_1       AM: Warmup (2 miles), 3x1000 with 3:30 recover...
Name: 0, dtype: object

In [41]:
vectorizer = CountVectorizer(ngram_range=(1,2))
X_feats = vectorizer.fit_transform(dat_small['notes'])

In [42]:
X_feats.shape

(407, 9575)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X_feats, dat_small['difficulty'], test_size=0.25)

In [77]:
mnb = MultinomialNB()
nb_model = mnb.fit(X_train, y_train)
y_pred = nb_model.predict(X_test)
#y_pred = mnb.fit(X_train, train['label']).predict(X_test)
print("Correct labels out of %d points : %d"
      % (X_test.shape[0],(y_test == y_pred).sum()))
print 'Accuracy:', float(sum(y_test == y_pred))/X_test.shape[0]
print 'Within 1:', float(sum((abs(y_test - y_pred) <= 1)))/X_test.shape[0]

Correct labels out of 102 points : 37
Accuracy: 0.362745098039
Within 1: 0.71568627451


In [78]:
results = pd.DataFrame(data = {'pred':y_pred, 'true':y_test}) #'diff':[abs(pred-true) for pred, true in zip(y_pred,y_test)]})

In [79]:
results.shape

(102, 2)

In [80]:
results[results['pred'] == 0]

,pred,true
77,0,0
80,0,0


In [81]:
res = results[(results['true'] != 0) & (results['pred'] != 0)]

In [82]:
print 'Accuracy:', float(sum(res['pred'] == res['true']))/res.shape[0]
print 'Within 1:', float(sum((abs(res['pred'] - res['true']) <= 1)))/res.shape[0]

Accuracy: 0.380434782609
Within 1: 0.75


In [45]:
y_probs = nb_model.predict_proba(X_test)

In [46]:
y_probs[0]

array([  1.30129555e-11,   8.62595128e-01,   3.06114407e-03,
         3.38376562e-03,   1.30956930e-01,   3.03221600e-06])

In [47]:
y_test.iloc[0]

1

In [63]:
i = 164
print dat_small['difficulty'].iloc[i]
dat_small['notes'].iloc[i]

5


u"Day off, feeling a bit sick (don't do too well with travel), so day off was good."

In [246]:
X_test

<102x9575 sparse matrix of type '<type 'numpy.int64'>'
	with 5730 stored elements in Compressed Sparse Row format>

In [248]:
accuracy = [None]*20
accuracy_within_1 = [None]*20
for j in range(20):  
    X_train, X_test, y_train, y_test = train_test_split(X_feats, dat_small['difficulty'], test_size=0.25, random_state=j)
    mnb = MultinomialNB()
    nb_model = mnb.fit(X_train, y_train)
    y_pred = nb_model.predict(X_test)
    #y_pred = mnb.fit(X_train, train['label']).predict(X_test)
    print("Correct labels out of %d points : %d"
          % (X_test.shape[0],(y_test == y_pred).sum()))
    print('Accuracy:', (y_test == y_pred).sum()/X_test.shape[0])
    accuracy[j] = (y_test == y_pred).sum()/X_test.shape[0]
    accuracy_within_1[j] = (abs(y_test - y_pred) <= 1).sum()/X_test.shape[0]

Correct labels out of 102 points : 41
Accuracy: 0.401960784314
Correct labels out of 102 points : 35
Accuracy: 0.343137254902
Correct labels out of 102 points : 37
Accuracy: 0.362745098039
Correct labels out of 102 points : 36
Accuracy: 0.352941176471
Correct labels out of 102 points : 38
Accuracy: 0.372549019608
Correct labels out of 102 points : 31
Accuracy: 0.303921568627
Correct labels out of 102 points : 43
Accuracy: 0.421568627451
Correct labels out of 102 points : 38
Accuracy: 0.372549019608
Correct labels out of 102 points : 38
Accuracy: 0.372549019608
Correct labels out of 102 points : 39
Accuracy: 0.382352941176
Correct labels out of 102 points : 31
Accuracy: 0.303921568627
Correct labels out of 102 points : 41
Accuracy: 0.401960784314
Correct labels out of 102 points : 40
Accuracy: 0.392156862745
Correct labels out of 102 points : 35
Accuracy: 0.343137254902
Correct labels out of 102 points : 35
Accuracy: 0.343137254902
Correct labels out of 102 points : 35
Accuracy: 0.34313

In [249]:
print(sum(accuracy)/len(accuracy))
print(sum(accuracy_within_1)/len(accuracy_within_1))

0.361764705882
0.75


In [250]:
dat.head()

,date,run_dist,notes,run_dist_1,notes_1
171,2016-10-10,10.0,"AM: Warmup (2 miles), 2 sets of (80, 50, 50), ...",10.0,"AM: Warmup (1.5 miles), Lift, Cooldown (2 mile..."
225,2016-11-21,10.0,"AM: Warmup (2 miles), 8x150 (Extensive Tempo),...",8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile..."
226,2016-11-22,8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile...",11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (..."
227,2016-11-23,11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (...",8.0,8 mile run. Not able to do squats or RDLs.
228,2016-11-24,8.0,8 mile run. Not able to do squats or RDLs.,10.5,"AM: Warmup (2 miles), 5x800 with 1:45 rest (Go..."


In [297]:
X_train_diff, X_test_diff, y_train_diff, y_test_diff = train_test_split(dat_small['notes'], dat_small['difficulty'], test_size=0.25)
X_train_work, X_test_work, y_train_work, y_test_work = train_test_split(dat_small['notes'], dat_small['is_workout'], test_size=0.25)

nb_model_diff = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), 
                     ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
nb_model_work = Pipeline([('vect', CountVectorizer(ngram_range=(1,2))), ('clf', MultinomialNB()),])
nb_model_diff.fit(X_train_diff, y_train_diff)
nb_model_work.fit(X_train_work, y_train_work)
#print(nb_model_diff.predict_proba(X_test_diff))
pred_diff = nb_model_diff.predict(X_test_diff)
pred_work = nb_model_work.predict(X_test_work)
print((sum(y_test_diff == pred_diff))/X_test_diff.shape[0])
print(sum((abs(y_test_diff - pred_diff) <= 1))/X_test_diff.shape[0])
print((sum(y_test_work == pred_work))/X_test_work.shape[0])

0.421568627451
0.813725490196
0.911764705882


In [298]:
dat.head()

,date,run_dist,notes,run_dist_1,notes_1
171,2016-10-10,10.0,"AM: Warmup (2 miles), 2 sets of (80, 50, 50), ...",10.0,"AM: Warmup (1.5 miles), Lift, Cooldown (2 mile..."
225,2016-11-21,10.0,"AM: Warmup (2 miles), 8x150 (Extensive Tempo),...",8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile..."
226,2016-11-22,8.0,"AM: Warmup (1.5 miles), Lift, Cooldown (1 mile...",11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (..."
227,2016-11-23,11.0,"AM: Warmup (2.5 miles), 8x300 with 2:30 rest (...",8.0,8 mile run. Not able to do squats or RDLs.
228,2016-11-24,8.0,8 mile run. Not able to do squats or RDLs.,10.5,"AM: Warmup (2 miles), 5x800 with 1:45 rest (Go..."


In [301]:
is_workout = nb_model_work.predict(dat['notes'])
is_workout_1 = nb_model_work.predict(dat['notes_1'])
difficulty = nb_model_diff.predict(dat['notes'])
difficulty_1 = nb_model_diff.predict(dat['notes_1'])

In [303]:
dat_knn = pd.DataFrame({'run_dist':dat['run_dist'], 'difficulty':difficulty, 'is_workout':is_workout,
                        'run_dist_1':dat['run_dist_1'], 'difficulty_1':difficulty_1, 'is_workout_1':is_workout_1})

In [306]:
dat_knn.iloc[100:105]

,difficulty,difficulty_1,is_workout,is_workout_1,run_dist,run_dist_1
219,1,1,1,0,9.00,5.00
220,1,4,0,1,5.00,7.00
221,4,1,1,1,7.00,7.00
224,1,1,1,0,4.50,10.25
227,1,1,0,1,10.25,9.00


In [305]:
dat_knn.shape

(1972, 6)

In [307]:
dat_knn.to_excel('dat_knn.xlsx')

In [1]:
def make_predictions(X, Y, val_X, val_Y, test_X, test_Y, s):
    # classifier = sklearn.svm.LinearSVC(n_jobs=2)
    classifier = sklearn.linear_model.LogisticRegression(n_jobs=2)
    # classifier = sklearn.naive_bayes.GaussianNB()
    # classifier = sklearn.svm.SVC(probability=True) # hard to scale to dataset with more than a couple of 10000 samples.

    total_data = X.shape[0]
    #subset for analysis
    # X = X[:round(total_data*s), :]
    # Y = Y[:round(total_data*s)]

    # print(s)
    # print("WITH {} TRAINING POINTS".format(X.shape[0]))

    classifier.fit(X, Y)
    test_y_hat = classifier.predict(test_X)
    results = sklearn.metrics.classification_report(test_Y, test_y_hat, digits=3)
    print('results: {0}'.format(results))
    accuracy_score = sklearn.metrics.accuracy_score(test_Y, test_y_hat)
    print('accuracy_score: {0}'.format(accuracy_score))

    roc_auc_score = sklearn.metrics.roc_auc_score(test_Y, test_y_hat)
    print('roc_auc_score: {0}'.format(roc_auc_score))

    # Plot ROC
    false_positive_rate, true_positive_rate, thresholds = sklearn.metrics.roc_curve(test_Y,
                                                                                    test_y_hat)
    return accuracy_score, roc_auc_score

In [3]:
'''
Text classification
'''
from __future__ import print_function
from __future__ import division
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import os
import sklearn
import sklearn.multiclass
import sklearn.svm
import sklearn.cross_validation
import sklearn.linear_model
import sklearn.naive_bayes
import sklearn.externals.joblib
import sklearn.metrics
import sklearn.feature_extraction.text
from nltk.tokenize import TreebankWordTokenizer
import utils
import argparse
import h5py
if utils.is_python3():
    import configparser as ConfigParser
else:
    import ConfigParser
    import scipy

In [4]:
print('The NumPy version is {0}.'.format(np.version.version))
print('The scikit-learn version is {0}.'.format(sklearn.__version__))
print('The SciPy version is {0}\n'.format(scipy.version.full_version))  # requires SciPy >= 0.16.0

def extract_features(X, val_X, test_X, max_ngram_size=2):
    print('max_ngram_size: {0}'.format(max_ngram_size))

    def make_text_list(X):

        return [' '.join([str(num) for num in x]) for x in X]

    print("Creating N-grams")
    X_texts = make_text_list(X)
    print("Creating N-grams for valid")
    val_X_texts = make_text_list(val_X)
    print("Creating N-grams for test")
    test_X_texts = make_text_list(test_X)
    all_texts = X_texts + val_X_texts + test_X_texts

    vect = sklearn.feature_extraction.text.CountVectorizer(ngram_range=(1, max_ngram_size),
                                                           tokenizer=TreebankWordTokenizer().tokenize)
    vect.fit(all_texts)  # build ngram dictionary
    X_features = vect.transform(X_texts)
    val_X_features = vect.transform(val_X_texts)
    test_X_features = vect.transform(test_X_texts)

    #print('features.shape: {0}'.format(features.shape))
    #np.savez(features_filepath, features=features)
    #np.savez("converted/X_features-"+str(max_ngram_size)+".npz", features=X_features)
    #np.savez("converted/val_X_features-"+str(max_ngram_size)+".npz", features=val_X_features)
    #np.savez("converted/test_X_features-"+str(max_ngram_size)+".npz", features=test_X_features)

    return X_features, val_X_features, test_X_features

The NumPy version is 1.13.3.
The scikit-learn version is 0.19.1.
The SciPy version is 1.0.0



In [5]:
def make_predictions(X, Y, val_X, val_Y, test_X, test_Y, s):
    # classifier = sklearn.svm.LinearSVC(n_jobs=2)
    classifier = sklearn.linear_model.LogisticRegression(n_jobs=2)
    # classifier = sklearn.naive_bayes.GaussianNB()
    # classifier = sklearn.svm.SVC(probability=True) # hard to scale to dataset with more than a couple of 10000 samples.

    total_data = X.shape[0]
    #subset for analysis
    # X = X[:round(total_data*s), :]
    # Y = Y[:round(total_data*s)]

    # print(s)
    # print("WITH {} TRAINING POINTS".format(X.shape[0]))

    classifier.fit(X, Y)
    test_y_hat = classifier.predict(test_X)
    results = sklearn.metrics.classification_report(test_Y, test_y_hat, digits=3)
    print('results: {0}'.format(results))
    accuracy_score = sklearn.metrics.accuracy_score(test_Y, test_y_hat)
    print('accuracy_score: {0}'.format(accuracy_score))

    roc_auc_score = sklearn.metrics.roc_auc_score(test_Y, test_y_hat)
    print('roc_auc_score: {0}'.format(roc_auc_score))

    # Plot ROC
    false_positive_rate, true_positive_rate, thresholds = sklearn.metrics.roc_curve(test_Y,
                                                                                    test_y_hat)
    return accuracy_score, roc_auc_score

In [6]:
feats = extract_features(train['notes'], val['notes'], test['notes'])

NameError: name 'train' is not defined

In [115]:
feats

(<1376x1677 sparse matrix of type '<type 'numpy.int64'>'
 	with 163919 stored elements in Compressed Sparse Row format>,
 <298x1677 sparse matrix of type '<type 'numpy.int64'>'
 	with 37148 stored elements in Compressed Sparse Row format>,
 <298x1677 sparse matrix of type '<type 'numpy.int64'>'
 	with 36130 stored elements in Compressed Sparse Row format>)

In [ ]:
feats = extract_features(tr_mini['notes'], v_mini['notes'], t_mini['notes'])